In [120]:
import mysql.connector 
import pandas as pd
from mysql.connector import Error
import numpy as np

In [68]:
def load_data_from_db(db_name):
    try:
        connection = mysql.connector.connect(host='127.0.0.1', database = 'ccf_data', user = 'root', password = '',autocommit=True)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
            df = pd.read_sql('SELECT * FROM {}'.format(db_name), con=connection)
            return df
    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
    

In [64]:
def insert_prediction_to_db(db_name="ccf_data", table_name="ccf_data_i", ):
    try :
        connection = mysql.connector.connect(host='127.0.0.1', database = 'ccf_data', user = 'root', password = '', autocommit=True)
        mySql_Insert_Table_Query = f"""INSERT INTO {db_name}.{table_name} SELECT * FROM ccf_data.ccf_data_to_add;  """
        cursor = connection.cursor()
        result_1 = cursor.execute(mySql_Insert_Table_Query)
        # We Will remove values from data_remaining table to avoid duplicated values 
        mySQL_Remove_Values_Query = f"""DELETE  FROM ccf_data_remaining WHERE ROUND(distance_from_home, 4) IN (SELECT ROUND(distance_from_home, 4) FROM ccf_data_to_add) AND ROUND(distance_from_last_transaction, 4) IN (SELECT ROUND(distance_from_last_transaction, 4) FROM ccf_data_to_add) AND ROUND(ratio_to_median_purchase_price, 4) IN (SELECT ROUND(ratio_to_median_purchase_price, 4) FROM ccf_data_to_add);"""
        result_4 = cursor.execute(mySQL_Remove_Values_Query)
        # Then we can discard predictions from ccf_data_to_add
        mySql_Delete_Table_Query = f"""DELETE FROM {db_name}.ccf_data_to_add;"""
        result_3 = cursor.execute(mySql_Delete_Table_Query)
        print("Update and Delete tables successfull")
        connection.commit()
        print(connection.autocommit)
    except mysql.connector.Error as error:
        print("Failed to create table in MySQL: {}".format(error))
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [39]:
def save_prediction_to_db(values, db_name = "ccf_data", table_name="ccf_data_to_add"):
    try :
        connection = mysql.connector.connect(host='127.0.0.1', database = db_name, user = 'root', password = '', autocommit=True)
        # 
        mySql_Create_Table_Query = f"""INSERT INTO {db_name}.{table_name} (distance_from_home, distance_from_last_transaction, ratio_to_median_purchase_price, repeat_retailer, user_chip, used_pin_number, online_order, fraud ) VALUES ({float(values["distance_from_home"])}, {float(values["distance_from_last_transaction"])}, {float(values["ratio_to_median_purchase_price"])}, {int(values["repeat_retailer"])}, {int(values["user_chip"])}, {int(values["used_pin_number"])}, {int(values["online_order"])}, {int(values["fraud"])});  """
        cursor = connection.cursor()
        # Il faut rajouter : si ça vient de la db print("salut ") sinon print('NTM')
        result = cursor.execute(mySql_Create_Table_Query)
        print("Laptop Table created successfully ")
        connection.commit()
        print(connection.autocommit)
    except mysql.connector.Error as error:
        print("Failed to create table in MySQL: {}".format(error))
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [65]:
insert_prediction_to_db()

Update and Delete tables successfull
True
MySQL connection is closed


In [151]:
save_prediction_to_db(values = df_full[-1001:-1000])

Laptop Table created successfully 
True
MySQL connection is closed


In [55]:
try:
    connection = mysql.connector.connect(host='127.0.0.1', database = 'ccf_data', user = 'root', password = '',autocommit=True)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        
except Error as e:
    print("Error while connecting to MySQL", e)

Connected to MySQL Server version  8.0.31
You're connected to database:  ('ccf_data',)


In [23]:
df_full = load_data_from_db("ccf_data_full")

Connected to MySQL Server version  8.0.31
You're connected to database:  ('ccf_data',)


/var/folders/2m/j3dvm9jx2_nb2cdjtp62_c340000gn/T/ipykernel_6629/1194592606.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM {}'.format(db_name), con=connection)


MySQL connection is closed


In [156]:
df_full.tail()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,user_chip,used_pin_number,online_order,fraud
999995,2.20710,0.112651,1.626800,1,1,0,0,0
999996,19.87270,2.683900,2.778300,1,1,0,0,0
999997,2.91486,1.472690,0.218075,1,1,0,1,0
999998,4.25873,0.242023,0.475822,1,0,0,1,0
999999,58.10810,0.318110,0.386920,1,1,0,1,0


In [62]:
round(df_full[["distance_from_home", "distance_from_last_transaction", "ratio_to_median_purchase_price"]], 4).duplicated().sum()

0

In [17]:
df_full.duplicated().sum()

0

In [69]:
df_partial = load_data_from_db("ccf_data_partial")

Connected to MySQL Server version  8.0.31
You're connected to database:  ('ccf_data',)
MySQL connection is closed


/var/folders/2m/j3dvm9jx2_nb2cdjtp62_c340000gn/T/ipykernel_6629/1194592606.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM {}'.format(db_name), con=connection)


In [88]:
df_partial.shape

(1000, 8)

In [84]:
df_i = load_data_from_db("ccf_data_i")
df_i.head()

Connected to MySQL Server version  8.0.31
You're connected to database:  ('ccf_data',)
MySQL connection is closed


/var/folders/2m/j3dvm9jx2_nb2cdjtp62_c340000gn/T/ipykernel_17324/1194592606.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM {}'.format(db_name), con=connection)


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,user_chip,used_pin_number,online_order,fraud
0,57.87790,0.311140,1.945940,1,1,0,0,0
1,10.82990,0.175591,1.294220,1,0,0,0,0
2,5.09108,0.805153,0.427715,1,0,0,1,0
3,2.24756,5.600040,0.362663,1,1,0,1,0
4,44.19090,0.566486,2.222770,1,1,0,1,0


In [70]:
df_i.shape

(1000, 8)

In [9]:
values = df_full[-1:]
values

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,user_chip,used_pin_number,online_order,fraud
999999,58.1081,0.31811,0.38692,1,1,0,1,0


In [60]:
try:
    connection = mysql.connector.connect(host='127.0.0.1', database = 'ccf_data', user = 'root', password = '',autocommit=True)
    mySql_Create_Table_Query = f"""INSERT INTO ccf_data.ccf_data_i (distance_from_home, distance_from_last_transaction, ratio_to_median_purchase_price, repeat_retailer, user_chip, used_pin_number, online_order, fraud ) VALUES (58.1081, 0.31811, 0.38692, 1, 1, 0, 1, 0);  """
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("Laptop Table created successfully ")
    connection.commit()
    print(connection.autocommit)
except mysql.connector.Error as error:
    print("Failed to create table in MySQL: {}".format(error))
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Laptop Table created successfully 
True
MySQL connection is closed


In [93]:
df_i = load_data_from_db("ccf_data_i")
df_i.tail()

Connected to MySQL Server version  8.0.31
You're connected to database:  ('ccf_data',)
MySQL connection is closed


/var/folders/2m/j3dvm9jx2_nb2cdjtp62_c340000gn/T/ipykernel_6629/1194592606.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT * FROM {}'.format(db_name), con=connection)


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,user_chip,used_pin_number,online_order,fraud
995,9.87342,1.022590,0.346643,1,0,0,0,0
996,168.09200,6.304360,0.416833,1,1,0,0,0
997,44.04760,0.510298,0.624706,1,0,0,1,0
998,2.99842,0.193681,0.743416,1,1,0,1,0
999,38.13340,0.167059,3.965640,1,0,0,1,0


In [89]:
insert_prediction_to_db(values = df_full[-2:-1])

Laptop Table created successfully 
True
MySQL connection is closed


In [163]:
df = pd.DataFrame(columns = ["distance_from_home", "distance_from_last_transaction", "ratio_to_median_purchase_price", "repeat_retailer", "user_chip", "used_pin_number", "online_order"])

In [102]:
df.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,user_chip,used_pin_number,online_order,fraud


In [164]:
df = df.append(pd.Series([2.91486, 1.472690, 0.218075, 1, 1, 0, 1], index = ["distance_from_home", "distance_from_last_transaction", "ratio_to_median_purchase_price", "repeat_retailer", "user_chip", "used_pin_number", "online_order"]), 
           ignore_index=True)

/var/folders/2m/j3dvm9jx2_nb2cdjtp62_c340000gn/T/ipykernel_6629/4202618930.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series([2.91486, 1.472690, 0.218075, 1, 1, 0, 1], index = ["distance_from_home", "distance_from_last_transaction", "ratio_to_median_purchase_price", "repeat_retailer", "user_chip", "used_pin_number", "online_order"]),


In [123]:
df =  pd.DataFrame(np.array([9.87342], [1.022590], [0.346643], [1], [0], [0], [0], [0]),columns = ["distance_from_home", "distance_from_last_transaction", "ratio_to_median_purchase_price", "repeat_retailer", "user_chip", "used_pin_number", "online_order"])

TypeError: array() takes from 1 to 2 positional arguments but 8 were given

In [162]:
df.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,user_chip,used_pin_number,online_order,fraud
0,2.91486,1.47269,0.218075,1.0,1.0,0.0,1.0,NaN


In [109]:
df.round(3).isin(df_i.round(3))

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,user_chip,used_pin_number,online_order,fraud
0,False,False,False,True,False,True,True,True


In [128]:
res = pd.merge(df_i, df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

# Display Result
print(res.shape[0])
print(df_i.shape[0])
print(res.shape[0] == df_i.shape[0])

999
1000
False


In [142]:
df["distance_from_home"][0]

9.87342

In [135]:
df_i

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,user_chip,used_pin_number,online_order,fraud
0,57.87790,0.311140,1.945940,1,1,0,0,0
1,10.82990,0.175591,1.294220,1,0,0,0,0
2,5.09108,0.805153,0.427715,1,0,0,1,0
3,2.24756,5.600040,0.362663,1,1,0,1,0
4,44.19090,0.566486,2.222770,1,1,0,1,0
...,...,...,...,...,...,...,...,...
995,9.87342,1.022590,0.346643,1,0,0,0,0
996,168.09200,6.304360,0.416833,1,1,0,0,0
997,44.04760,0.510298,0.624706,1,0,0,1,0
998,2.99842,0.193681,0.743416,1,1,0,1,0


In [186]:
df_full[(df_full["distance_from_home"].round(4) == df["distance_from_home"][0].round(4)) & (df_full["distance_from_last_transaction"].round(4) == df["distance_from_last_transaction"][0].round(4))&(df_full["ratio_to_median_purchase_price"].round(4) == df["ratio_to_median_purchase_price"][0].round(4))&(df_full["repeat_retailer"].round(4) == df["repeat_retailer"][0].round(4))&(df_full["user_chip"].round(4) == df["user_chip"][0].round(4))&(df_full["used_pin_number"].round(4) == df["used_pin_number"][0].round(4))&(df_full["online_order"].round(4) == df["online_order"][0].round(4))]["fraud"].iloc[0]

0

In [177]:
df_full.loc[999997].round(4)

distance_from_home                2.9149
distance_from_last_transaction    1.4727
ratio_to_median_purchase_price    0.2181
repeat_retailer                   1.0000
user_chip                         1.0000
used_pin_number                   0.0000
online_order                      1.0000
fraud                             0.0000
Name: 999997, dtype: float64

In [178]:
df.iloc[0].round(4)

distance_from_home                2.9149
distance_from_last_transaction    1.4727
ratio_to_median_purchase_price    0.2181
repeat_retailer                   1.0000
user_chip                         1.0000
used_pin_number                   0.0000
online_order                      1.0000
Name: 0, dtype: float64